# Cloud Environment
This tutorial demonstrates the configuration and use of a simple BSK-RL environment considering cloud coverage. The satellite has to image targets while managing its battery level. Additionally, reward is inversely proportional to the amount of cloud coverage. Still, the satellite cannot observe the true cloud coverage of each target, only its forecast.

## Load Modules

In [ ]:
import gymnasium as gym
import numpy as np
from typing import Optional

from Basilisk.architecture import bskLogging
from Basilisk.utilities import orbitalMotion
from bsk_rl import act, obs, sats
from bsk_rl.sim import dyn, fsw
from bsk_rl.utils.orbital import random_orbit
from bsk_rl.scene.targets import UniformTargets
from bsk_rl.data.unique_image_data import (
    UniqueImageData,
    UniqueImageStore,
    UniqueImageReward,
)

bskLogging.setDefaultLogLevel(bskLogging.BSK_WARNING)

## Configure the Satellite
* [Observations](../api_reference/obs/index.rst): 
    - SatProperties: Body angular velocity, instrument pointing direction, body position, body velocity, battery charge (properties in [flight software model](../api_reference/sim/fsw/index.rst) or [dynamics model](../api_reference/sim/dyn/index.rst)). Also, customized dynamics property in CustomDynModel below: Angle between the sun and the solar panel.
    - OpportunityProperties: Target's priority, cloud coverage forecast, and standard deviation of cloud coverage forecast (upcoming 32 targets). Also, time until the opportunity to ground station opens and closes.
    - Time: Simulation time.
    - Eclipse: Next eclipse start and end times. 
* [Actions](../api_reference/act/index.rst):
    - Charge: Enter a sun-pointing charging mode for 60 seconds.
    - Image: Image target from upcoming 32 targets
* [Dynamics model](../api_reference/sim/dyn/index.rst): FullFeaturedDynModel is used and a property, angle between sun and solar panel, is added.
* [Flight software model](../api_reference/sim/fsw/index.rst): SteeringImagerFSWModel is used.

In [ ]:
class CustomSatComposed(sats.ImagingSatellite):
    observation_spec = [
        obs.SatProperties(
            dict(prop="omega_BP_P", norm=0.03),
            dict(prop="c_hat_P"),
            dict(prop="r_BN_P", norm=orbitalMotion.REQ_EARTH * 1e3),
            dict(prop="v_BN_P", norm=7616.5),
            dict(prop="battery_charge_fraction"),
            dict(prop="solar_angle_norm"),
        ),
        obs.OpportunityProperties(
            # dict(fn=lambda sat, opp: print(opp)),
            dict(prop="opportunity_open", norm=5700),
            dict(prop="opportunity_close", norm=5700),
            type="ground_station",
            n_ahead_observe=1,
        ),
        obs.Eclipse(),
        obs.OpportunityProperties(
            dict(prop="priority"),
            dict(fn=lambda sat, opp: opp["object"].cloud_cover_forecast),
            dict(fn=lambda sat, opp: opp["object"].cloud_cover_sigma),
            type="target",
            n_ahead_observe=32,
        ),
        obs.Time(),
    ]

    action_spec = [
        act.Charge(duration=60.0),
        act.Image(n_ahead_image=32),
    ]

    class CustomDynModel(dyn.FullFeaturedDynModel):
        @property
        def solar_angle_norm(self) -> float:
            sun_vec_N = (
                self.world.gravFactory.spiceObject.planetStateOutMsgs[
                    self.world.sun_index
                ]
                .read()
                .PositionVector
            )
            sun_vec_N_hat = sun_vec_N / np.linalg.norm(sun_vec_N)
            solar_panel_vec_B = np.array([0, 0, -1])  # Not default configuration
            mat = np.transpose(self.BN)
            solar_panel_vec_N = np.matmul(mat, solar_panel_vec_B)
            error_angle = np.arccos(np.dot(solar_panel_vec_N, sun_vec_N_hat))

            return error_angle / np.pi

    dyn_type = CustomDynModel
    fsw_type = fsw.SteeringImagerFSWModel

When instantiating a satellite, these parameters can be overriden with a constant or 
rerandomized every time the environment is reset using the ``sat_args`` dictionary.

In [ ]:
dataStorageCapacity = 20 * 8e6 * 100
sat_args = CustomSatComposed.default_sat_args(
    oe=random_orbit,
    imageAttErrorRequirement=0.01,
    imageRateErrorRequirement=0.01,
    batteryStorageCapacity=80.0 * 3600 * 2,
    storedCharge_Init=lambda: np.random.uniform(0.4, 1.0) * 80.0 * 3600 * 2,
    u_max=0.2,
    K1=0.5,
    nHat_B=np.array([0, 0, -1]),
    imageTargetMinimumElevation=np.radians(45),
    rwBasePower=20,
    maxWheelSpeed=1500,
    storageInit=lambda: np.random.randint(
        0 * dataStorageCapacity,
        0.01 * dataStorageCapacity,
    ),  # Initialize storage use close to zero
    wheelSpeeds=lambda: np.random.uniform(
        -1, 1, 3
    ),  # Initialize reaction wheel speeds close to zero
)

# Make the satellites
satellites = []
satellites.append(
    CustomSatComposed(
        "EO",
        sat_args,
    )
)

## Making a Scenario with Cloud Covered Targets
Using UniformTargets as a base, attach the following information to each target:

* `true_cloud_cover` represents the true cloud coverage. Information from external sources, such as historical cloud data, can be used here based on each target's position.

* `cloud_cover_forecast` represents the cloud coverage forecast. Forecast from external sources can be plugged in here.

* `cloud_cover_sigma` represents the standard deviation of the cloud coverage forecast.

In [ ]:
class CloudTargets(UniformTargets):
    mu_data = 0.6740208166434426
    sigma_max = 0.05
    sigma_min = 0.01

    def regenerate_targets(self) -> None:
        super().regenerate_targets()
        for target in self.targets:
            target.true_cloud_cover = np.clip(
                np.random.uniform(0.0, self.mu_data * 2), 0.0, 1.0
            )
            target.cloud_cover_sigma = np.random.uniform(self.sigma_min, self.sigma_max)
            target.cloud_cover_forecast = np.clip(
                np.random.normal(target.true_cloud_cover, target.cloud_cover_sigma),
                0.0,
                1.0,
            )


n_targets = (1000, 10000)
scenario = CloudTargets(n_targets=n_targets)

## Adding a Filter Based on Cloud Coverage Forecast
It is possible to add a filter to the satellite using [add_access_filter](../api_reference/sats/index.rst) to remove targets with `cloud_cover_forecast` higher than a threshold from the observations.

In [ ]:
def cloud_cover_filter(opportunity):
    if opportunity["type"] == "target":
        return True if opportunity["object"].cloud_cover_forecast < 0.2 else False
    return True


# Uncomment the following line to add the filter to the satellite
# satellites[0].add_access_filter(cloud_cover_filter)

## Making a Rewarder Considering Cloud Coverage
A linear reward model is considered, where the reward is proportional to the cloud coverage of the target until a given threshold given by `cloud_threshold`. It has similar settings as the [UniqueImageReward](../api_reference/data/index.rst) class, but `cloud_covered` and `cloud_free` information is added. Additionally, the `calculate_reward` function is modified for the linear reward model.

In [ ]:
from typing import TYPE_CHECKING

if TYPE_CHECKING:  # pragma: no cover
    from bsk_rl.scene.targets import (
        Target,
    )


class CloudImagePercentData(UniqueImageData):
    """DataType for unique images of targets."""

    def __init__(
        self,
        imaged: Optional[set["Target"]] = None,
        duplicates: int = 0,
        known: Optional[set["Target"]] = None,
        cloud_covered: Optional[set["Target"]] = None,
        cloud_free: Optional[set["Target"]] = None,
    ) -> None:
        """Construct unit of data to record unique images.

        Keeps track of ``imaged`` targets, a count of ``duplicates`` (i.e. images that
        were not rewarded due to the target already having been imaged), and all
        ``known`` targets in the environment.

        Args:
            imaged: Set of targets that are known to be imaged.
            duplicates: Count of target imaging duplication.
            known: Set of targets that are known to exist (imaged and unimaged).
            cloud_covered: Set of imaged targets that are known to be cloud covered.
            cloud_free: Set of imaged targets that are known to be cloud free.
        """
        super().__init__(imaged=imaged, duplicates=duplicates, known=known)
        if cloud_covered is None:
            cloud_covered = set()
        if cloud_free is None:
            cloud_free = set()
        self.cloud_covered = set(cloud_covered)
        self.cloud_free = set(cloud_free)

    def __add__(self, other: "CloudImagePercentData") -> "CloudImagePercentData":
        """Combine two units of data.

        Args:
            other: Another unit of data to combine with this one.

        Returns:
            Combined unit of data.
        """

        imaged = self.imaged | other.imaged
        duplicates = (
            self.duplicates
            + other.duplicates
            + len(self.imaged)
            + len(other.imaged)
            - len(imaged)
        )
        known = self.known | other.known

        cloud_covered = self.cloud_covered | other.cloud_covered
        cloud_free = self.cloud_free | other.cloud_free

        return self.__class__(
            imaged=imaged,
            duplicates=duplicates,
            known=known,
            cloud_covered=cloud_covered,
            cloud_free=cloud_free,
        )


class CloudImagePercentDataStore(UniqueImageStore):
    """DataStore for unique images of targets."""

    data_type = CloudImagePercentData

    def compare_log_states(
        self, old_state: np.ndarray, new_state: np.ndarray
    ) -> CloudImagePercentData:
        """Check for an increase in logged data to identify new images.

        Args:
            old_state: older storedData from satellite storage unit
            new_state: newer storedData from satellite storage unit

        Returns:
            list: Targets imaged at new_state that were unimaged at old_state
        """
        data_increase = new_state - old_state
        if data_increase <= 0:
            return CloudImagePercentData()
        else:
            assert self.satellite.latest_target is not None
            self.update_target_colors([self.satellite.latest_target])

            cloud_coverage = self.satellite.latest_target.true_cloud_cover
            cloud_threshold = 0.7
            if cloud_coverage > cloud_threshold:
                cloud_covered = [self.satellite.latest_target]
                cloud_free = []
            else:
                cloud_covered = []
                cloud_free = [self.satellite.latest_target]
            return CloudImagePercentData(
                imaged={self.satellite.latest_target},
                cloud_covered=cloud_covered,
                cloud_free=cloud_free,
            )


class CloudImagingPercentRewarder(UniqueImageReward):
    """DataManager for rewarding unique images."""

    data_store_type = CloudImagePercentDataStore

    def calculate_reward(
        self, new_data_dict: dict[str, CloudImagePercentData]
    ) -> dict[str, float]:
        """Reward new each unique image once using self.reward_fn().

        Args:
            new_data_dict: Record of new images for each satellite

        Returns:
            reward: Cumulative reward across satellites for one step
        """
        reward = {}
        imaged_counts = {}
        for new_data in new_data_dict.values():
            for target in new_data.imaged:
                if target not in imaged_counts:
                    imaged_counts[target] = 0
                imaged_counts[target] += 1

        for sat_id, new_data in new_data_dict.items():
            reward[sat_id] = 0.0
            for target in new_data.cloud_free:
                if target not in self.data.imaged:
                    reward[sat_id] += self.reward_fn(
                        target.priority,
                        target.true_cloud_cover,
                        imaged_counts[target],
                    )
        return reward


# Define the reward function as a function of the priority of the target, the cloud cover, and the number of times the target has been imaged
def reward_function(priority, cloud_cover, count_target):
    cloud_threshold = 0.7
    return priority / count_target * (1 - cloud_cover / cloud_threshold)


rewarder = CloudImagingPercentRewarder(reward_fn=reward_function)

## Initializing and Interacting with the Environment
For this example, we will be using the single-agent [SatelliteTasking](../api_reference/index.rst) 
environment. Along with passing the satellite that we configured, the environment takes
a [scenario](../api_reference/scene/index.rst), which defines the environment the
satellite is acting in, and a [rewarder](../api_reference/data/index.rst), which defines
how data collected from the scenario is rewarded.

In [ ]:
env = gym.make(
    "GeneralSatelliteTasking-v1",
    satellites=satellites,
    terminate_on_time_limit=True,
    scenario=scenario,
    rewarder=rewarder,
    sim_rate=0.5,
    max_step_duration=300.0,
    time_limit=95 * 60 * 3,
    log_level="INFO",
    failure_penalty=0,
    # disable_env_checker=True,  # For debugging
)

First, reset the environment. It is possible to specify the seed when resetting the environment.

In [ ]:
observation, info = env.reset(seed=1)

It is possible to printing out the actions and observations. The composed satellite [action_description](../api_reference/sats/index.rst) returns a human-readable action map each satellite has the same action space and similar observation space.

In [ ]:
print("Actions:", satellites[0].action_description)
print("States:", env.unwrapped.satellites[0].observation_description, "\n")

# Using the composed satellite features also provides a human-readable state:
for satellite in env.unwrapped.satellites:
    for k, v in satellite.observation_builder.obs_dict().items():
        print(f"{k}:  {v}")

Then, run the simulation until timeout or agent failure.

In [ ]:
count = 0
while True:
    if count == 0:
        # Vector with an action for each satellite (we can pass different actions for each satellite)
        # Tasking all satellites to charge (tasking None as the first action will raise a warning)
        action_vector = [0]
    elif count == 1:
        # None will continue the last action, but will also raise a warning
        action_vector = [None]
    elif count == 2:
        # Tasking different actions for each satellite
        action_vector = [1]
    else:
        # Tasking random actions
        action_vector = env.action_space.sample()
    count += 1

    observation, reward, terminated, truncated, info = env.step(action_vector)

    # Show the custom normalized observation vector
    # print("\tObservation:", observation)

    if terminated or truncated:
        print("Episode complete.")
        break

After the running the simulation, we can check the reward, number of imaged targets that were covered by clouds and that were not covered by clouds (according to the threshold set in the rewarder).

In [ ]:
print("Total reward:", env.unwrapped.rewarder.cum_reward)
print("Covered by clouds:", env.unwrapped.rewarder.data.cloud_covered)
print("Not covered by clouds:", env.unwrapped.rewarder.data.cloud_free)